In [1]:
%load_ext lab_black

In [2]:
import math

import numpy as np
import torch
import torch.nn as nn
from torch.optim import SGD

In [3]:
class Predictor(nn.Module):
    def __init__(self, inputDim, hiddenDim, outputDim):
        super(Predictor, self).__init__()

        self.rnn = nn.LSTM(
            input_size=inputDim, hidden_size=hiddenDim, batch_first=True
        )
        self.output_layer = nn.Linear(hiddenDim, outputDim)

    def forward(self, inputs, hidden0=None):
        output, (hidden, cell) = self.rnn(inputs, hidden0)
        output = self.output_layer(output[:, -1, :])

        return output

In [4]:
def mkDataSet(data_size, data_length=50, freq=60.0, noise=0.00):
    """
    params\n
    data_size : データセットサイズ\n
    data_length : 各データの時系列長\n
    freq : 周波数\n
    noise : ノイズの振幅\n
    returns\n
    train_x : トレーニングデータ（t=1,2,...,size-1の値)\n
    train_t : トレーニングデータのラベル（t=sizeの値）\n
    """
    train_x = []
    train_t = []

    for offset in range(data_size):
        train_x.append(
            [
                [
                    math.sin(2 * math.pi * (offset + i) / freq)
                    + np.random.normal(loc=0.0, scale=noise)
                ]
                for i in range(data_length)
            ]
        )
        train_t.append([math.sin(2 * math.pi * (offset + data_length) / freq)])

    return train_x, train_t

In [5]:
def mkRandomBatch(train_x, train_t, batch_size=10):
    """
    train_x, train_tを受け取ってbatch_x, batch_tを返す。
    """
    batch_x = []
    batch_t = []

    for _ in range(batch_size):
        idx = np.random.randint(0, len(train_x) - 1)
        batch_x.append(train_x[idx])
        batch_t.append(train_t[idx])

    return torch.tensor(batch_x), torch.tensor(batch_t)

In [6]:
def main():
    training_size = 10000
    test_size = 1000
    epochs_num = 100
    hidden_size = 5
    batch_size = 100

    train_x, train_t = mkDataSet(training_size)
    test_x, test_t = mkDataSet(test_size)

    model = Predictor(1, hidden_size, 1)
    criterion = nn.MSELoss()
    optimizer = SGD(model.parameters(), lr=0.01)

    for epoch in range(epochs_num):
        # training
        running_loss = 0.0
        training_accuracy = 0.0
        for i in range(int(training_size / batch_size)):
            optimizer.zero_grad()

            data, label = mkRandomBatch(train_x, train_t, batch_size)

            output = model(data)

            loss = criterion(output, label)
            loss.backward()
            optimizer.step()

            running_loss += loss.data.item()
            training_accuracy += np.sum(
                np.abs((output.data - label.data).numpy()) < 0.1
            )

        # test
        test_accuracy = 0.0
        for i in range(int(test_size / batch_size)):
            offset = i * batch_size
            data, label = (
                torch.tensor(test_x[offset : offset + batch_size]),
                torch.tensor(test_t[offset : offset + batch_size]),
            )
            output = model(data, None)

            test_accuracy += np.sum(
                np.abs((output.data - label.data).numpy()) < 0.1
            )

        training_accuracy /= training_size
        test_accuracy /= test_size

        print(
            "%d loss: %.3f, training_accuracy: %.5f, test_accuracy: %.5f"
            % (epoch + 1, running_loss, training_accuracy, test_accuracy)
        )

In [7]:
if __name__ == "__main__":
    main()

1 loss: 50.462, training_accuracy: 0.06750, test_accuracy: 0.06700
2 loss: 39.416, training_accuracy: 0.06840, test_accuracy: 0.08300
3 loss: 25.802, training_accuracy: 0.09060, test_accuracy: 0.11800
4 loss: 12.317, training_accuracy: 0.12940, test_accuracy: 0.13600
5 loss: 5.584, training_accuracy: 0.13050, test_accuracy: 0.13600
6 loss: 3.781, training_accuracy: 0.14560, test_accuracy: 0.15300
7 loss: 3.274, training_accuracy: 0.16010, test_accuracy: 0.18600
8 loss: 2.950, training_accuracy: 0.16830, test_accuracy: 0.15200
9 loss: 2.671, training_accuracy: 0.16330, test_accuracy: 0.16800
10 loss: 2.531, training_accuracy: 0.16820, test_accuracy: 0.16800
11 loss: 2.370, training_accuracy: 0.18180, test_accuracy: 0.21900
12 loss: 2.176, training_accuracy: 0.21080, test_accuracy: 0.21800
13 loss: 2.098, training_accuracy: 0.24100, test_accuracy: 0.25200
14 loss: 1.996, training_accuracy: 0.24600, test_accuracy: 0.25200
15 loss: 1.873, training_accuracy: 0.29190, test_accuracy: 0.31600
